In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1] #dydx = z

    y_derivs[1] = -1*y[0] #dzdx = -y

    return y_derivs

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)

    x_ipoh = xi + 0.5*h
    
    x_ipo = xi + h
    
    y_temp = np.zeros(nv)
    
    #k1 = hf(xn, yn)
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #k2 = hf(xn + (1/2)*h, yn + (1/2)*k1)
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #k3 = hf(xn + (1/2)*h, yn + (1/2)*k2)
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #k4 = hf(xn + h, yn + k3)
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    #ynp1 = yn + (1/6)k1 + (1/3)k2 + (1/3)K3 + (1/6)k4
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

In [ ]:
def rk6ck_mv_core(dydx, xi, yi, nv, h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)

    x_ipoh = xi + 0.5*h
    
    x_ipo = xi + h
    
    y_temp = np.zeros(nv)
    
    #k1 = hf(xn, yn)
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #k2 = hf(xn + (1/2)*h, yn + (1/2)*k1)
    y_temp[:] = yi[:] + 0.2*k1[:]
    y_derivs = dydx(xi + 0.2*h,y_temp)
    k2[:] = h*y_derivs[:]
    
    #k3 = hf(xn + (1/2)*h, yn + (1/2)*k2)
    y_temp[:] = yi[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(xi + 0.3,y_temp)
    k3[:] = h*y_derivs[:]
    
    #k4 = hf(xn + h, yn + k3)
    y_temp[:] = yi[:] + 0.3*k1[:] - 0.9*k2[:] + (6/5)*k3[:]
    y_derivs = dydx(xi + 0.6,y_temp)
    k4[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] - (11/54)*k1[:] + (5/2)*k2[:] - (70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(xi + 1,y_temp)
    k5[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] - (575/13824)*k3[:] + (44275/110592)*k4[:] + \
        (253/4096)*k5[:]
    y_derivs = dydx(xi + (7/8),y_temp)
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    #ynp1 = yn + (1/6)k1 + (1/3)k2 + (1/3)K3 + (1/6)k4
    #yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    yip1 = yi + (37/378)*k1 + (0)*k2 + (250/621)*k3 + (125/594)*k4 + (0)*k5 + (512/1771)*k6
    yip1s = yi + (2825/27648)*k1 + (0)*k2 + (18575/48384)*k3 + (13525/55296)*k4 + (277/14336)*k5 + (1/4)*k6
    
    return yip1, yip1s

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 10000
    
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    h_step = h 
    
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking one step of size h vs two septs of h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_i,nv,0.5*h_step)
        
        #compute error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our step is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        #check iteration 
        if(i>=imax):
            print("too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        i+=1
        
    #next time, try a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    return y_2, h_new, h_step

In [ ]:
def rk6ck_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 10000
    
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    h_step = h 
    
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking one step of size h vs two septs of h/2
        #y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        #y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        #y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_i,nv,0.5*h_step)
        yip1, yip1s = rk6ck_mv_core(dydx, x_i, y_i, nv, h_step)
        
        #compute error
        Delta = np.fabs(yip1 - yip1s) #np.fabs(y_2 - y_11)
        
        #if the error is too large take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our step is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        #check iteration 
        if(i>=imax):
            print("too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        i+=1
        
    #next time, try a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    return yip1, h_new, h_step

In [ ]:
def rk4_mv(dfdx,a,b,y_a,tol):
    
    
    xi = a
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)
    
    imax = 10000
    
    i = 0 
    
    #set a number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions 
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set flag
    flag = 1
    
    #loop until we reac the right side
    while(flag):
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_1+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0
        
        #update the values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays 
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y 
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            #output some info
            s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
            print(s)
        
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i += 1 
        
        #break if new xi is == b 
        if(xi==b):
            #output some info
            s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i + 1,xi, h_step, b)
            print(s)
            flag = 0
        
    #return asnwer
    return x,y

In [ ]:
def rk6ck_mv(dfdx,a,b,y_a,tol):
    
    
    xi = a
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)
    
    imax = 10000
    
    i = 0 
    
    #set a number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions 
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set flag
    flag = 1
    
    #loop until we reac the right side
    while(flag):
        yi_new, h_new, h_step = rk6ck_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_1+1
            yi_new, h_new, h_step = rk6ck_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0
        
        #update the values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays 
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y 
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            #output some info
            s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
            print(s)
        
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i += 1 
        
        #break if new xi is == b 
        if(xi==b):
            #output some info
            s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i + 1,xi, h_step, b)
            print(s)
            flag = 0
        
    #return asnwer
    return x,y

# perform the integration 

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#x,y = rk4_mv(dydx, a, b, y_0, tolerance)
x,y = rk6ck_mv(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x, y[:,0],'o',label='y(x)')
plt.plot(x, y[:,1],'o',label='dydx(x)')

xx = np.linspace(0,2*np.pi, 1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(frameon=False)

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dxdy_error = (y[:,1]-cosine)

plt.plot(x, y_error, label='y(x) Error')
plt.plot(x, dxdy_error, label='dxdy(x) Error')
plt.legend(frameon=False)